In [1]:
import numpy as np
import datasets
import pickle

from collections import Counter
from unicode_substitutions import unicode_pairs, char_dict
import my_datasets

In [2]:
len(char_dict)

28

In [3]:
char_dict

{'a': 'а',
 'c': 'ϲ',
 'e': 'е',
 'g': 'ɡ',
 'i': 'і',
 'j': 'ϳ',
 'o': 'ο',
 'p': 'р',
 's': 'ѕ',
 'x': 'х',
 'y': 'у',
 'A': 'Α',
 'B': 'Β',
 'C': 'Ϲ',
 'E': 'Ε',
 'H': 'Η',
 'I': 'Ι',
 'J': 'Ј',
 'K': 'Κ',
 'M': 'Μ',
 'N': 'Ν',
 'O': 'Ο',
 'P': 'Ρ',
 'S': 'Ѕ',
 'T': 'Τ',
 'X': 'Χ',
 'Y': 'Υ',
 'Z': 'Ζ'}

In [3]:
ds_name = 'pile_100M'
frac = 10
# watermark tokens that appear 1-n times
cutoff = 3

In [4]:
ds = my_datasets.get_dataset(ds_name)

Loading cached shuffled indices for dataset at /home/ryan/haveibeentrainedon/data/pile1e8_orig/cache-db4e2c3fb658967d.arrow


In [5]:
len(ds)

57586

In [6]:
# it's possible that we are perturbing duplicated sequences
control_idx = int(frac * 0.01 * len(ds))
control_idx

5758

In [7]:
subset = ds.select(range(control_idx))

In [8]:
text = ''.join(subset['text'])
np.mean([ len(i.split(' ')) for i in subset['text'] ])

# words per sentence

1105.2575547064953

In [9]:
c = Counter(text.split(' '))
alpha_numeric = list(filter(lambda x: x[0].isalnum(), c.most_common()))
alpha_numeric[:10], len(alpha_numeric)

([('the', 267910),
  ('of', 147531),
  ('and', 127065),
  ('to', 126813),
  ('a', 104553),
  ('in', 87392),
  ('is', 55283),
  ('that', 49304),
  ('for', 45881),
  ('with', 38073)],
 122807)

In [17]:
l = [ (i,count) for i,count in alpha_numeric if count > 1 ]
len(l)
l[-100:]

[('MICU', 2),
 ('Macaulay', 2),
 ('Maroscia', 2),
 ('equidimensional', 2),
 ('socle', 2),
 ('betw', 2),
 ('Weaken', 2),
 ('Yanaulsky', 2),
 ('CareerBuilder', 2),
 ('GeForce', 2),
 ('75W', 2),
 ('GTX950', 2),
 ('promos', 2),
 ('MUSIC', 2),
 ('baba', 2),
 ('VegByte', 2),
 ('Inhumane', 2),
 ('derogation', 2),
 ('equated', 2),
 ('premolar', 2),
 ('eunuchs', 2),
 ('Transgender', 2),
 ('Outrankify', 2),
 ('Niikuni', 2),
 ('onna', 2),
 ('kayoiji', 2),
 ('basho', 2),
 ('abusutorakushon', 2),
 ('PASSAGE', 2),
 ('Intertextual', 2),
 ('Interrelational', 2),
 ('Kinya', 2),
 ('Meiji', 2),
 ('Akutagawa', 2),
 ('Joryū', 2),
 ('Kyū', 2),
 ('incest', 2),
 ('bestiality', 2),
 ('yuan', 2),
 ('Parody', 2),
 ('Translated', 2),
 ('languidly', 2),
 ('downy', 2),
 ('reproachful', 2),
 ('extraterrestrials', 2),
 ('Extraterrestrials', 2),
 ('madwoman', 2),
 ('detested', 2),
 ('kuru', 2),
 ('towa', 2),
 ('Nō', 2),
 ('Kanze', 2),
 ('Beauties', 2),
 ('Grandfather', 2),
 ('spiderweb', 2),
 ('kimono', 2),
 ('trepida

In [19]:
one_to_n = list(filter(lambda x: x[1] > control_idx and x[1] < control_idx*cutoff, alpha_numeric))
one_to_n

[('or', 8395),
 ('we', 7703),
 ('had', 7373),
 ('were', 7175),
 ('his', 6702),
 ('but', 6266),
 ('he', 6139),
 ('her', 6075),
 ('which', 5684),
 ('can', 5638),
 ('my', 5432),
 ('has', 5286),
 ('their', 4937),
 ('will', 4929),
 ('one', 4924),
 ('all', 4900),
 ('been', 4780),
 ('they', 4628),
 ('if', 4584),
 ('would', 4230),
 ('more', 4219),
 ('she', 4199),
 ('your', 3939),
 ('about', 3932),
 ('into', 3823),
 ('out', 3655),
 ('when', 3649),
 ('our', 3590),
 ('its', 3519),
 ('than', 3492),
 ('so', 3430),
 ('no', 3350),
 ('other', 3347),
 ('like', 3282),
 ('up', 3254),
 ('could', 3245),
 ('any', 3191),
 ('what', 3189),
 ('also', 3187),
 ('there', 3163),
 ('some', 3148),
 ('In', 3128),
 ('do', 3037),
 ('only', 3016),
 ('who', 2948),
 ('said', 2909),
 ('It', 2898)]

In [24]:
# two kinds of dictionaries
word_pairs = []
for original, j in one_to_n:
    substitute = ''.join([ char_dict.get(c, c) for c in original ])
    if substitute != original:
        word_pairs.append((original, substitute))     
word_pairs

[('or', 'οr'),
 ('we', 'wе'),
 ('had', 'hаd'),
 ('were', 'wеrе'),
 ('his', 'hіѕ'),
 ('he', 'hе'),
 ('her', 'hеr'),
 ('which', 'whіϲh'),
 ('can', 'ϲаn'),
 ('my', 'mу'),
 ('has', 'hаѕ'),
 ('their', 'thеіr'),
 ('will', 'wіll'),
 ('one', 'οnе'),
 ('all', 'аll'),
 ('been', 'bееn'),
 ('they', 'thеу'),
 ('if', 'іf'),
 ('would', 'wοuld'),
 ('more', 'mοrе'),
 ('she', 'ѕhе'),
 ('your', 'уοur'),
 ('about', 'аbοut'),
 ('into', 'іntο'),
 ('out', 'οut'),
 ('when', 'whеn'),
 ('our', 'οur'),
 ('its', 'іtѕ'),
 ('than', 'thаn'),
 ('so', 'ѕο'),
 ('no', 'nο'),
 ('other', 'οthеr'),
 ('like', 'lіkе'),
 ('up', 'uр'),
 ('could', 'ϲοuld'),
 ('any', 'аnу'),
 ('what', 'whаt'),
 ('also', 'аlѕο'),
 ('there', 'thеrе'),
 ('some', 'ѕοmе'),
 ('In', 'Ιn'),
 ('do', 'dο'),
 ('only', 'οnlу'),
 ('who', 'whο'),
 ('said', 'ѕаіd'),
 ('It', 'Ιt')]

In [25]:
pickle.dump(word_pairs, open('./all_unicode.pkl', 'wb'))